<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Agrupaciones <em>k</em>-means (Clustering)</font></h1>


## Introducción

Existen muchos modelos de agrupación. En este laboratorio representaremos el modelo considerado el mas simple de entre todos. A pesar de su simplicidad, _k_-means es muy usado para realizar agrupaciones en muchas aplicaciones de ciencia de datos, es especialmente util si usted necesita descubrir rapidamente cosas importantes en datos sin etiquetar.

Algunas aplicaciones reales de _k_-means incluyen:

-   segmentación de clientes
-   entender que tratan de hacer los visitantes de un sitio web
-   reconocimiento de patrones
-   compresión de información

En este laboratorio aprendera a agrupar usando _k_-means con 3 ejemplos:

-   usar _k_-means en un conjunto de datos generado aleatoriamente
-   usar _k_-means para la segmentación de clientes


## Indice

1.  <a href="#item1">Usar <em>k</em>-means en un conjunto de datos generado aleatoriamente</a>    
2.  <a href="#item2">Usar <em>k</em>-means para la segmentación de clientes</a> 


Antes de empezar con el contenido, descarguemos todas las dependencias necesarias.


In [3]:
import random # librería para generar números aleatorios
import numpy as np  # librería para manejar datos vectorizados
import pandas as pd # librería para procesar datos como dataframes 

import matplotlib.pyplot as plt # librería para graficar 
# backend para graficar diagramas en el explorador
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

print('Libraries imported.')

Libraries imported.


C:\Users\Ramiro Salaberry\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


<a id='item1'></a>


## 1. _k_-means en un Conjunto de Datos Generado Aleatoriamente


Demostremos como _k_-means trabaja utilizando un ejemplo con datos diseñados


#### 30 puntos de datos pertenecientes a 2 distintas agrupaciones (x1 es la primer característica y x2 la segunda)


In [ ]:
# datos
x1 = [-4.9, -3.5, 0, -4.5, -3, -1, -1.2, -4.5, -1.5, -4.5, -1, -2, -2.5, -2, -1.5, 4, 1.8, 2, 2.5, 3, 4, 2.25, 1, 0, 1, 2.5, 5, 2.8, 2, 2]
x2 = [-3.5, -4, -3.5, -3, -2.9, -3, -2.6, -2.1, 0, -0.5, -0.8, -0.8, -1.5, -1.75, -1.75, 0, 0.8, 0.9, 1, 1, 1, 1.75, 2, 2.5, 2.5, 2.5, 2.5, 3, 6, 6.5]

print('Datapoints defined!')

#### Definir una función que asigne cada dato a una agrupación


In [ ]:
colors_map = np.array(['b', 'r'])
def assign_members(x1, x2, centers):
    compare_to_first_center = np.sqrt(np.square(np.array(x1) - centers[0][0]) + np.square(np.array(x2) - centers[0][1]))
    compare_to_second_center = np.sqrt(np.square(np.array(x1) - centers[1][0]) + np.square(np.array(x2) - centers[1][1]))
    class_of_points = compare_to_first_center > compare_to_second_center
    colors = colors_map[class_of_points + 1 - 1]
    return colors, class_of_points

print('assign_members function defined!')

#### Definir una función que actualice el centroide de cada agrupación


In [ ]:
# actualizar la media 
def update_centers(x1, x2, class_of_points):
    center1 = [np.mean(np.array(x1)[~class_of_points]), np.mean(np.array(x2)[~class_of_points])]
    center2 = [np.mean(np.array(x1)[class_of_points]), np.mean(np.array(x2)[class_of_points])]
    return [center1, center2]

print('assign_members function defined!')

#### Deinir una función que grafique cada dato junto con los centroides de la agrupación


In [ ]:
def plot_points(centroids=None, colors='g', figure_title=None):
    # graficar la figura
    fig = plt.figure(figsize=(15, 10))  # crear un objeto figura 
    ax = fig.add_subplot(1, 1, 1)
    
    centroid_colors = ['bx', 'rx']
    if centroids:
        for (i, centroid) in enumerate(centroids):
            ax.plot(centroid[0], centroid[1], centroid_colors[i], markeredgewidth=5, markersize=20)
    plt.scatter(x1, x2, s=500, c=colors)
    
    # definir las marcas
    xticks = np.linspace(-6, 8, 15, endpoint=True)
    yticks = np.linspace(-6, 6, 13, endpoint=True)

    # arreglar el eje horizontal
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)

    # agregar etiquetas a las marcas
    xlabels = xticks
    ax.set_xticklabels(xlabels)
    ylabels = yticks
    ax.set_yticklabels(ylabels)

    # dar estilo a las marcas
    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')
    ax.tick_params('both', length=2, width=1, which='major', labelsize=15)
    
    # añadir etiquetas a los ejes
    ax.set_xlabel('x1', fontsize=20)
    ax.set_ylabel('x2', fontsize=20)
    
    # añadir un título a la figura
    ax.set_title(figure_title, fontsize=24)

    plt.show()

print('plot_points function defined!')

#### Inicializar la gráfica de los puntos con _k_-means


In [ ]:
plot_points(figure_title='Scatter Plot of x2 vs x1')

#### Iniciar las agrupaciones definidas aleatoriamente con _k_-means y añadirlas a la gráfica


In [ ]:
centers = [[-2, 2], [2, -2]]
plot_points(centers, figure_title='k-means Initialization')

#### Ejecutar _k_-means (4 iteraciones unicamente)


In [ ]:
number_of_iterations = 4
for i in range(number_of_iterations):
    input('Iteration {} - Press Enter to update the members of each cluster'.format(i + 1))
    colors, class_of_points = assign_members(x1, x2, centers)
    title = 'Iteration {} - Cluster Assignment'.format(i + 1)
    plot_points(centers, colors, figure_title=title)
    input('Iteration {} - Press Enter to update the centers'.format(i + 1))
    centers = update_centers(x1, x2, class_of_points)
    title = 'Iteration {} - Centroid Update'.format(i + 1)
    plot_points(centers, colors, figure_title=title)

Hemos observado hasta ahora como trabaja <em>k</em>-means. Veamos otro ejemplo ahora con muchos mas datos. Para esto, utilizaremos la librería <strong>random</strong> para generar miles de datos.


### Generar los Datos


Primero, necesitaremos configurar la semilla aleatoria. Usaremos la función de Numpy **random.seed()** y estableceremos la semilla (seed) en 0. En otras palabras,  **random.seed(0)**.


In [ ]:
np.random.seed(0)

A continuación crearemos _agrupaciones aleatorias_ de los datos utilizando la clase **make_blobs**. Esta clase puede tomar muchas entradas pero solo usaremos las especificadas.

<b> <u> Entrada </u> </b>

<ul>
    <li> <b>n_samples</b>: Número total de datos divididos equitativamente entre las agrupaciones. </li>
    <ul> <li> El valor sera de: 5000 </li> </ul>
    <li> <b>centers</b>: Número de centros a generar, o las ubicaciones conocidas de estos. </li>
    <ul> <li> El valor sera de: [[4, 4], [-2, -1], [2, -3],[1,1]] </li> </ul>
    <li> <b>cluster_std</b>: Desviación estandar de las agrupaciones. </li>
    <ul> <li> El valor sera de: 0.9 </li> </ul>
</ul>

<b> <u> Salida </u> </b>

<ul>
    <li> <b>X</b>: Arreglo para la forma [n_samples, n_features]. (Matriz de Características)</li>
    <ul> <li> Las muestras generadas. </li> </ul> 
    <li> <b>y</b>: Arreglo para la forma [n_samples]. (Vector de Respuesta)</li>
    <ul> <li> Las etiquetas de enteros para los miembros de las agrupaciones de cada muestra. </li> </ul>
</ul>


In [ ]:
X, y = make_blobs(n_samples=5000, centers=[[4, 4], [-2, -1], [2, -3], [1, 1]], cluster_std=0.9)

Mostrar el diagrama de dispersión de los datos generados aleatoriamente.


In [ ]:
plt.figure(figsize=(15, 10))
plt.scatter(X[:, 0], X[:, 1], marker='.')

### Configurar _k_-means


Ahora que hemos generado nuestros datos, configuremos la agrupación con _k_-means


La clase KMeans tiene muchos parámetros a usar pero solo utilizaremos los siguientes tres:

<ul>
    <li> <strong>init</strong>: Método para inicializar los centroides. </li>
    <ul>
        <li> El valor será: "k-means++". k-means++ selecciona centros de agrupación iniciales para <em>k</em>-means de una forma inteligente para acelerar la convergencia. </li>
    </ul>
    <li> <strong>n_clusters</strong>: Número de agrupaciones a formar así como el número de centroides a generar. </li>
    <ul> <li>  El valor será: 4 (debido a que tenemos 4 centros)</li> </ul>
    <li> <strong>n_init</strong>: Número de veces que el algoritmo de <em>k</em>-means será ejecutado con distintas semillas de centroides. Los resultados finales eran la mejor salida de n_init ejecuciones consecutivas en términos de inercia. </li>
    <ul> <li> El valor será: 12 </li> </ul>
</ul>

Inicializar KMeans con estos parámetros, donde el parámetro de salida es **k_means**. 


In [ ]:
k_means = KMeans(init="k-means++", n_clusters=4, n_init=12)

Ajustemos el modelo KMeans con la matriz de características <b> X </b> creada anteriormente.


In [ ]:
k_means.fit(X)

Tomemos las etiquetas para cada punto del modelo usando el atributo **.labels\_** de KMeans y guardelo como **k_means_labels**.


In [ ]:
k_means_labels = k_means.labels_
k_means_labels

También tomaremos las coordenadas de los centros de la agrupación utilizando **.cluster_centers\_** de KMeans, las guardaremos como **k_means_cluster_centers**.


In [ ]:
k_means_cluster_centers = k_means.cluster_centers_
k_means_cluster_centers

### Visualizar las Agrupaciones Resultantes


Ahora que hemos generado aleatoriamente los datos y el modelo KMeans esta inicializado, grafiquemoslos y veamos como quedó la agrupación.


Lea por favor el siguiente código y los comentarios para entender como graficar el modelo.


In [ ]:
# incializar la gráfica con las dimensiones especificadas.
fig = plt.figure(figsize=(15, 10))

# colors utiliza un mapa de colores, el cual producirá un arreglo de colores con base
# al número de etiqueta. Utilzaremos set(k_means_labels) para obtener las 
# etiquetas unicas.
colors = plt.cm.Spectral(np.linspace(0, 1, len(set(k_means_labels))))

# generar una gráfica
ax = fig.add_subplot(1, 1, 1)

# haga un ciclo a traves de los datos y grafique los puntos y centroides.
# k estará en el rango de 0 a 3 lo cual emparejará el número de agrupaciones en el conjunto de datos.
for k, col in zip(range(len([[4,4], [-2, -1], [2, -3], [1, 1]])), colors):

    # crear una lista de todos los puntos, donde los que estan en
    # la agrupación estan etiquetados como verdaderos y los que no
    # como falsos
    my_members = (k_means_labels == k)
    
    # definir el centroide o centro de la agrupación
    cluster_center = k_means_cluster_centers[k]
    
    # graficar los puntos con color col.
    ax.plot(X[my_members, 0], X[my_members, 1], 'w', markerfacecolor=col, marker='.')
    
    # graficar los centroides con el color especificado pero con contorno mas oscuro
    ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,  markeredgecolor='k', markersize=6)

# título de la gráfica
ax.set_title('KMeans')

# remover las marcas del eje x
ax.set_xticks(())

# remover las marcas del eje y
ax.set_yticks(())

# mostrar la gráfica
plt.show()

<a id='item2'></a>


## Usar _k_-means para la Segmentación de Clientes


Imagine que tiene un conjunto de datos sobre un cliente y esta interesado en saber sobre su comportamiento haciendo uso de sus datos hístoricos.

La segmentación de clientes es la práctica de dividir la información de la base de clientes en grupos de individuos con características parecidas. Es una estrategia significativa ya que un negocio puede seleccionar estos grupos de clientes y transladar en base a ellos recursos de marketing. Por ejemplo, un grupo podría contener clientes que se inclinan por las altas ganancias y bajo riesgo, esto es, son mas proclives a comprar productos o suscribirse a algun servicio. La tarea del negocio es mantener estos clientes. Otro grupo podría incluir clientes de organizaciones sin ánimo de lucro, etc.


### Descargar los Datos


Descarguemos los datos y vamos a guardarlos en un archivo CSV llamado **customer_segmentation.csv**


In [8]:
!wget -q -O 'customer_segmentation.csv' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/customer_segmentation.csv
print('Data downloaded!')

Data downloaded!


"wget" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


Ahora que los datos han sido descargados, los pondremos en un dataframe _pandas_.


In [9]:
customers_df = pd.read_csv('customer_segmentation.csv')
customers_df.head()

FileNotFoundError: [Errno 2] File customer_segmentation.csv does not exist: 'customer_segmentation.csv'

### Pre procesado


Como puede observar, en este conjunto de datos **Address** es una variable categórica. El algoritmo k-means no es directamente aplicable a variables categóricas porque la función de la distancia euclidiana no es realmente significativa para variables discretas. Quitemos esta característica para ejecutar el agrupamiento.


In [ ]:
df = customers_df.drop('Address', axis=1)
df.head()

Normalicemos el conjunto de datos. Pero, ¿Por que necesitamos normalizar en primer lugar? La normalización es un método estadístico que ayuda a los algoritmos matemáticos a interpretar características con distintas magnitudes y distribuciones por igual. Usaremos **StandardScaler()** para normalizar el conjunto de datos.


In [ ]:
from sklearn.preprocessing import StandardScaler

X = df.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

### Modelado


Ejecutemos nuestro modelo y agrupemos nuestros clientes en 3 agrupaciones.


In [ ]:
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

<h2 id="insights">Perspectivas</h2>


Observe que cada fila en el conjunto de datos representa un cliente, por consiguiente, a cada fila se le asigna una etiqueta.


In [ ]:
df["Labels"] = labels
df.head(5)

Podemos revisar facilmente los valores del centroide promediando las características de cada agrupación.


In [ ]:
df.groupby('Labels').mean()

<em>k</em>-means particionará sus clientes en tres grupos debido a que hemos especificado que se generaran 3 agrupaciones. Los clientes en cada agrupación son parecidos entre ellos en términos de las características incluidas en el conjunto de datos.

Ahora podemos generar un perfil para cada grupo, considerando las características comunes de cada agrupación. Por ejemplo, las 3 agrupaciones pueden ser:

Now we can create a profile for each group, considering the common characteristics of each cluster. 
For example, the 3 clusters can be:

-   OLDER, HIGH INCOME, E INDEBTED
-   MIDDLE AGED, MIDDLE INCOME, Y FINANCIALLY RESPONSIBLE
-   YOUNG, LOW INCOME, E INDEBTED


Sin embargo usted puede idear sus propios perfiles basandose en las medias de arriba y proponer etiquetas que considere mejor describen a la agrupación.


Espero que haya podido ver el poder de _k_-means. Este algoritmo de agrupación nos ofrece perspectivas interesantes del conjunto de datos y nos permitió agrupar los datos en tres agrupaciones. Quiza se hubiera alcanzado los mismos resultados con multiples pruebas y experimentos.


### Gracias por terminar este laboratorio

Este cuaderno fue creado por [Saeed Aghabozorgi](https://ca.linkedin.com/in/saeedaghabozorgi?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701ES-Coursera-21685115&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701ES-Coursera-21685115&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ) y [Alex Aklson](https://www.linkedin.com/in/aklson?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701ES-Coursera-21685115&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701ES-Coursera-21685115&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). Esperamos que haya encontrado este laboratorio de su interes y educativo. Tómese la libertad de contactar conmigo para cualquier duda o aclaración.


Este cuaderno forma parte del curso en **Coursera** llamado _Applied Data Science Capstone_. Si accede a este cuaderno desde afuera del curso, puede tomarlo en línea haciendo clic [aquí](http://cocl.us/DP0701EN_Coursera_Week3_LAB1).


<hr>

Copyright © 2018 [Cognitive Class](https://cognitiveclass.ai?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701ES-Coursera-21685115&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701ES-Coursera-21685115&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
